In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
import os
import datetime as dt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

# Collect data

In [ ]:
data = []

# filepaths for all the data for each season since 20-21
filepaths = []
for i in range(38):
    filepaths.append(
        Path(f'../../../season20_21/data/fpl/data_week{i+1}.csv')
    )
    filepaths.append(
        Path(f'../../../season21_22/data/fpl/data_week{i+1}.csv')
    )
    filepaths.append(
        Path(f'../../../season22_23/data/fpl_data/fpl_week_{i+1}.csv')
    )

for filepath in filepaths:
    gameweek_data = pd.read_csv(filepath, index_col=0)
    modification_time = os.path.getmtime(filepath)
    modification_datetime = dt.datetime.fromtimestamp(modification_time)
    gameweek_data['modification_datetime'] = modification_datetime
    data.append(gameweek_data)

display(len(data))

In [ ]:
pd.concat(data, ignore_index=True)

In [ ]:
# all data for season 20-21
data = []
for i in range(38):
    filepath = Path(f'../../../season20_21/data/fpl/data_week{i+1}.csv')
    gameweek_data = pd.read_csv(filepath, index_col=0)
    gameweek_data['gameweek'] = i+1
    data.append(gameweek_data)

df = pd.concat(data, ignore_index=True)
display(df.head())
display(df.shape)

In [ ]:
xg_data = []
xga_data = []
for ix, row in df.iterrows():
    my_gameweek = row['gameweek']
    xg_data.append( row[f'xG_week{my_gameweek}'] )
    xga_data.append( row[f'xGA_week{my_gameweek}'] )

df['gameweek_xG'] = xg_data
df['gameweek_xGA'] = xga_data

display(df.head())

In [ ]:
columns_20_21 = ['assists', 'bonus', 'bps', 'clean_sheets', 'corners_and_indirect_freekicks_order', 'creativity', 'creativity_rank', 
                 'creativity_rank_type', 'direct_freekicks_order', 'dreamteam_count', 'element_type', 'event_points', 'first_name', 
                 'goals_conceded', 'goals_scored', 'ict_index', 'ict_index_rank', 'ict_index_rank_type', 'influence', 'influence_rank',
                 'influence_rank_type', 'minutes', 'now_cost', 'own_goals', 'penalties_missed', 'penalties_order', 'penalties_saved',
                 'points_per_game', 'red_cards', 'saves', 'second_name', 'selected_by_percent', 'threat', 'threat_rank', 'threat_rank_type',
                 'total_points', 'web_name', 'yellow_cards', 'team_name', 'xG', 'xA', 'gameweek', 'gameweek_xG', 'gameweek_xGA']

In [ ]:
df20_21 = df[columns_20_21].copy()
df20_21

In [ ]:
df20_21[df20_21.web_name=='Aubameyang'].head()

Need to write a script that
- extracts xGA from correct column based on gameweek
- finds the opponent (team) for each gameweek, calculates their relevant metrics coming into the game and takes double gameweeks etc. into account

In [ ]:
# FPL fixtures needed to get correct gameweeks for each match

filepath = Path(f'../../../season20_21/data/fpl/fixtures.csv')
fixtures = pd.read_csv(filepath, index_col=0)
fixtures['kickoff_time'] = pd.to_datetime(fixtures.kickoff_time)

display(fixtures.head())
display(fixtures.shape)

In [ ]:
# teams needed to map fixture team_ids to actual team names

filepath = Path(f'../../../season20_21/data/fpl/teams.csv')
teams = pd.read_csv(filepath, index_col=0)
display(teams)

In [ ]:
# map team_id to actual name
team_id_dict = dict(zip(teams.id.values, teams.name.values))
display(team_id_dict)

In [ ]:
fixtures['away_team'] = fixtures['team_a'].apply(lambda x: team_id_dict[x])
fixtures['home_team'] = fixtures['team_h'].apply(lambda x: team_id_dict[x])
display(fixtures.head())

In [ ]:
# FBREF data needed for xg values
filepath = Path(f'../../../season20_21/data/fbref/fixtures20_21.csv')
fixtures_fbref = pd.read_csv(filepath, index_col=0)

#drop null rows
fixtures_fbref = fixtures_fbref[~fixtures_fbref.isnull().all(axis=1)] 

fixtures_fbref['Date'] = pd.to_datetime(fixtures_fbref['Date'])

display(fixtures_fbref.head())
display(fixtures_fbref.shape)

In [ ]:
# map fbref team names to fpl team names
team_name_dict = dict(zip(np.sort(fixtures_fbref.Home.unique()), np.sort(fixtures.home_team.unique())))
display(team_name_dict)

In [ ]:
fixtures_fbref['Home'] = fixtures_fbref['Home'].apply(lambda x: team_name_dict[x])
fixtures_fbref['Away'] = fixtures_fbref['Away'].apply(lambda x: team_name_dict[x])
fixtures_fbref.head()

In [ ]:
# get xg data to fpl fixtures df

home_xg = []
away_xg = []
for ix, row in fixtures.iterrows():
    home_team = row.home_team
    away_team = row.away_team
    home_team_xg = fixtures_fbref.loc[(fixtures_fbref['Home']==home_team) & (fixtures_fbref['Away']==away_team), 'xG home'].values[0]
    away_team_xg = fixtures_fbref.loc[(fixtures_fbref['Home']==home_team) & (fixtures_fbref['Away']==away_team), 'xG away'].values[0]
    home_xg.append( home_team_xg )
    away_xg.append( away_team_xg )

fixtures['xg_home'] = home_xg
fixtures['xg_away'] = away_xg

display(fixtures.head())

In [ ]:
fixtures.isnull().sum()

In [ ]:
# get each team's data on one row
fixtures_melt = fixtures.melt(id_vars=['xg_home', 'xg_away', 'team_h_score', 'team_a_score', 'event', 'kickoff_time', 'id'], value_vars=['home_team', 'away_team'])

# get team's xG (home xG if at home, away xG if at an away game)
fixtures_melt['xG'] = fixtures_melt.apply(lambda x: x['xg_home'] if x['variable']=='home_team' else x['xg_away'], axis=1)
fixtures_melt['xGA'] = fixtures_melt.apply(lambda x: x['xg_away'] if x['variable']=='home_team' else x['xg_home'], axis=1)

# sort by date
fixtures_melt = fixtures_melt.sort_values(by='kickoff_time').reset_index(drop=True)

# calculate rolling averages
#fixtures_melt['xG_rolling_10'] = fixtures_melt[['value','xG']].groupby(by='value').rolling(10).mean().reset_index().sort_values(by='level_1')['xG'].values
for i in [5,9,18,38]:
    fixtures_melt[f'xG_ewm_{i}'] = fixtures_melt[['value','xG']].groupby(by='value').ewm(alpha=1/i).mean().reset_index().sort_values(by='level_1')['xG'].values
    fixtures_melt[f'xGA_ewm_{i}'] = fixtures_melt[['value','xGA']].groupby(by='value').ewm(alpha=1/i).mean().reset_index().sort_values(by='level_1')['xGA'].values

fixtures_melt

In [ ]:
fixtures_melt[fixtures_melt.value=='Fulham'].head()

In [ ]:
df20_21.head()

In [ ]:
# columns to be fetched from team data
col_names = [f'xG_ewm_{i}' for i in [5,9,18,38]]
col_names += [f'xGA_ewm_{i}' for i in [5,9,18,38]]
nr_cols = len(col_names) 
team_data = []
opponent_data = []
for ix, row in df20_21.iterrows():
    gameweek = row.gameweek
    team = row.team_name
    games = fixtures_melt[(fixtures_melt.value==team) & (fixtures_melt.event==gameweek)]
    if games.shape[0]!=1:
        team_data.append( np.array([np.nan]*nr_cols) )
        opponent_data.append( np.array([np.nan]*nr_cols) )
    elif games.shape[0]==1:
        # add team data
        team_data.append( games[col_names].values.flatten() )
        # find opponent data
        home_game = games.variable.values[0]=='home_team'
        game_id = games.id.values[0]
        if home_game:
            opponent_team = fixtures.loc[(fixtures.home_team==team) & (fixtures.event==gameweek), 'away_team'].values[0]
        else:
            opponent_team = fixtures.loc[(fixtures.away_team==team) & (fixtures.event==gameweek), 'home_team'].values[0]
        opponent_games = fixtures_melt[(fixtures_melt.value==opponent_team) & (fixtures_melt.event==gameweek) & (fixtures_melt.id==game_id)]
        # add opponent data
        opponent_data.append( opponent_games[col_names].values.flatten() )
    else:
        print(f'Check number of games for ix {ix}!')
     

new_col_names = ['team_'+col for col in col_names]
team_data_df = pd.DataFrame(team_data, columns=new_col_names)
new_oppo_col_names = ['opponent_'+col for col in col_names]
opponent_data_df = pd.DataFrame(opponent_data, columns=new_oppo_col_names)

df20_21 = df20_21.join([team_data_df, opponent_data_df])

display(df20_21.head())
display(df20_21.tail())
display(df20_21.shape)

In [ ]:
cols_list = ['minutes', 'xG']
cols_list += [f'xG_week{i+1}' for i in range(3)] 
cols_list += [f'xGA_week{i+1}' for i in range(3)] 
cols_list += ['gameweek_xG']
cols_list += ['gameweek_xGA']
aux = df.loc[df['web_name'].str.contains('Aubameyang'), cols_list].copy()
aux['gameweek_minutes'] = aux['minutes'].diff().fillna(aux['minutes'])
aux['gameweek_xG_aux'] = aux['xG'].diff().fillna(aux['xG'])

display(aux)

In [ ]:
# all data for season 21-22
data = []
for i in range(38):
    filepath = Path(f'../../../season21_22/data/fpl/data_week{i+1}.csv')
    gameweek_data = pd.read_csv(filepath, index_col=0)
    modification_time = os.path.getmtime(filepath)
    modification_datetime = datetime.datetime.fromtimestamp(modification_time)
    gameweek_data['modification_datetime'] = modification_datetime
    data.append(gameweek_data)

data = pd.concat(data, ignore_index=True)
display(data.head())

In [ ]:
# all data for season 22-23
for i in range(38):
    filepath = Path(f'../../../../season22_23/data/fpl_data/fpl_week_{i+1}.csv')
    gameweek_data = pd.read_csv(filepath, index_col=0)
    modification_time = os.path.getmtime(filepath)
    modification_datetime = datetime.datetime.fromtimestamp(modification_time)
    gameweek_data['modification_datetime'] = modification_datetime
    data.append(gameweek_data)